# First notebook to try out Cython

### In this notebook I look into implementing the dot product with pure python and then start adding some cython in the mix.

The dot product of a vector is defined as:

$$u \cdot v =  \sum_{n=1}^{N} u_i * v_i $$

In [1]:
# load Cython extension magic to work in notebook
%load_ext Cython

In [2]:
# define the size of the vector
N = int(1e6)

In [3]:
# python random library to generate random vectors
import random

In [4]:
# generate to vectors as Python lists
u = [random.random() for x in range(N)]
v = [random.random() for x in range(N)]

In [5]:
# define the dot product in pure Python
def dot_product(a, b):
    # sum up elementwise product
    return sum([a_ * b_ for a_, b_ in zip(a, b)])

### Let's first see how fast python can go

In [ ]:
%%timeit -r 7 -n 100
dot_product(u,v)

### And now let Cython do all it's magic 

In [ ]:
%%cython --annotate
def dot_product_cython(a, b): 
    return sum([i * j for i, j in zip(a, b)])

In [ ]:
%%timeit -r 7 -n 100
dot_product_cython(u, v)

## Not a bad start, Cython manages to run in about two thirds of the time of the pure Python implementation

## But now to the real power of Cython with type declarations and static memory allocations

In [9]:
%%cython --annotate

# import malloc and free 
# for those not familiar with C, malloc allocates the requested memory and free releases it
from libc.stdlib cimport malloc, free

# Sidenote:
# in C "vanilla" arrays are defined using pointers
# in C++ 11 the standard library provides containers,
# such as vector<> which does all the memory management

# the actual dot product calculation returns a scalar
# to be able to compare with numpy we use double precision
cdef double dot_product_cython_1(double *a, double *b, int size):
    # declare sum as a double with value 0
    cdef double sum = 0.0
    cdef int i = 0
    for i in range(size):
        # add up elementwise product of a and b
        sum+= a[i]*b[i]
    # and return the sum
    return sum

# this is a helper function that copies Python lists to C arrays 
def call_cython(list a, list b):
    
    # get the size of list a, let's assume b is the same size
    # as we don't perform this check in the other functions
    cdef size_t size = len(a)
    
    # allocate memory of a_ and b_ C arrays that will contain a copy of a and b lists, respectively
    cdef double *a_ = <double *> malloc(size*sizeof(double *))
    cdef double *b_ = <double *> malloc(size*sizeof(double *))
    cdef double result
    
    # one of the things that slows down python for loops is that it 
    # tries to dynamically determine the type of each element of the iterator
    # so we can tell Cython that i is an int
    cdef int i
    # copy python list to C array
    for i in range(size):
        a_[i] = a[i]
        b_[i] = b[i]
        i+=1
    
    # calculate the actual dot product
    result = dot_product_cython_1(a_, b_, size)
    
    # and very important! Release the memory
    free(a_)
    free(b_)
    
    return result

# So was it worth it?

In [9]:
%%timeit -r 3 -n 100
call_cython(u, v)

NameError: name 'call_cython' is not defined

In [11]:
import numpy as np

In [12]:
%%timeit
np.dot(u, v)

36.6 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Much faster than numpy! But is this implementation correct?

In [13]:
np.isclose(call_cython(u, v), np.dot(u, v))

True

# Cython provides an easy to use interface that allows a developer to mix C-like syntax with Python with low production time and impressive speed gains! 